<a href="https://colab.research.google.com/github/Khaimin6/CTF_solution/blob/main/14_zipfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://ctf.hackme.quest/static/zipfile.zip

--2026-01-01 09:00:29--  https://ctf.hackme.quest/static/zipfile.zip
Resolving ctf.hackme.quest (ctf.hackme.quest)... 114.44.204.174
Connecting to ctf.hackme.quest (ctf.hackme.quest)|114.44.204.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320244 (313K) [application/zip]
Saving to: ‘zipfile.zip’

zipfile.zip         100%[===================>] 312.74K   613KB/s    in 0.5s    

2026-01-01 09:00:30 (613 KB/s) - ‘zipfile.zip’ saved [320244/320244]



In [ ]:
## There might be some silent update on challenge
## 2024.09.22 - 2408 KB
## 2026.01.01 - 313 KB  (added extra layers zipfile.zip/zipfile.zip/...)

# from zipfile import ZipFile, ZipInfo
# from io import BytesIO

# def load_zip(data):
#     if isinstance(data, bytes):
#         data = BytesIO(data)
#     return ZipFile(data)

# def unzip(data):
#     file = load_zip(data)
#     return [file.read(c) for c in file.infolist()]

# path = "./zipfile.zip"
# target = load_zip(path)
# result = [[] for i in range(184)]

# for flag in target.infolist():

#     fname = flag.filename[-18:-3]   # 0/1/0/../1/1ÿPK
#     num = ''.join(fname.split('/')) # 010..11
#     index = int(num, 2)             # 0b010..11

#     xor = bytes()

#     for sub2 in unzip(*unzip(target.read(flag))):
#         xor = sub2 if (not xor) else bytes(i^j for i,j in zip(xor,sub2))
#     result[index].append(xor)

# print(b"".join(b for a, b, c in result))

In [ ]:
## Warning: decompress using tools will cause error
## For this challenge,
## `7z x <path>` - path too long error
## `7z e <path>` - remove duplicate file

# from genericpath import isfile
# while isfile("zipfile.zip"):
#     !7z e zipfile.zip -y >/dev/null

In [ ]:
from zipfile import ZipFile, ZipInfo
from io import BytesIO

def load_zip(data):
    if isinstance(data, bytes):
        data = BytesIO(data)
    return ZipFile(data)

def unzip(data, return_content=True):
    file: ZipFile = load_zip(data)
    items = file.infolist()
    if file.filename: print("[UNZIP]", file.filename)

    if len(items) == 1:
        return file.read(items[0])

    print(f"[RETURN] {len(items)} items")

    if return_content:
        return [(c.filename, file.read(c)) for c in items]  #(filename, content)
    else:
        return items

In [ ]:
stream = unzip("./zipfile.zip")

while isinstance(stream, bytes):
    stream = unzip(stream)

print(stream)


In [ ]:
set(path[-20:] for path, content in stream)

{'0/0/0/0/0/0/0/0/0ÿPK',
 '0/0/0/0/0/0/0/0/1ÿPK',
 '0/0/0/0/0/0/0/1/0ÿPK',
 '0/0/0/0/0/0/0/1/1ÿPK',
 '0/0/0/0/0/0/1/0/0ÿPK',
 '0/0/0/0/0/0/1/0/1ÿPK',
 '0/0/0/0/0/0/1/1/0ÿPK',
 '0/0/0/0/0/0/1/1/1ÿPK',
 '0/0/0/0/0/1/0/0/0ÿPK',
 '0/0/0/0/0/1/0/0/1ÿPK',
 '0/0/0/0/0/1/0/1/0ÿPK',
 '0/0/0/0/0/1/0/1/1ÿPK',
 '0/0/0/0/0/1/1/0/0ÿPK',
 '0/0/0/0/0/1/1/0/1ÿPK',
 '0/0/0/0/0/1/1/1/0ÿPK',
 '0/0/0/0/0/1/1/1/1ÿPK',
 '0/0/0/0/1/0/0/0/0ÿPK',
 '0/0/0/0/1/0/0/0/1ÿPK',
 '0/0/0/0/1/0/0/1/0ÿPK',
 '0/0/0/0/1/0/0/1/1ÿPK',
 '0/0/0/0/1/0/1/0/0ÿPK',
 '0/0/0/0/1/0/1/0/1ÿPK',
 '0/0/0/0/1/0/1/1/0ÿPK',
 '0/0/0/0/1/0/1/1/1ÿPK',
 '0/0/0/0/1/1/0/0/0ÿPK',
 '0/0/0/0/1/1/0/0/1ÿPK',
 '0/0/0/0/1/1/0/1/0ÿPK',
 '0/0/0/0/1/1/0/1/1ÿPK',
 '0/0/0/0/1/1/1/0/0ÿPK',
 '0/0/0/0/1/1/1/0/1ÿPK',
 '0/0/0/0/1/1/1/1/0ÿPK',
 '0/0/0/0/1/1/1/1/1ÿPK',
 '0/0/0/1/0/0/0/0/0ÿPK',
 '0/0/0/1/0/0/0/0/1ÿPK',
 '0/0/0/1/0/0/0/1/0ÿPK',
 '0/0/0/1/0/0/0/1/1ÿPK',
 '0/0/0/1/0/0/1/0/0ÿPK',
 '0/0/0/1/0/0/1/0/1ÿPK',
 '0/0/0/1/0/0/1/1/0ÿPK',
 '0/0/0/1/0/0/1/1/1ÿPK',


In [ ]:
for _path, content in stream:
    print(unzip(unzip(content)))

In [ ]:
# target = load_zip(stream)
result = [[] for i in range(184)]

for _path, content in stream:

    fname = _path[-18:-3]           # 0/1/0/../1/1ÿPK
    num = ''.join(fname.split('/')) # 010..11
    index = int(num, 2)             # 0b010..11

    xor = bytes()

    for _, sub2 in unzip(unzip(content)):
        xor = sub2 if (not xor) else bytes(i^j for i,j in zip(xor,sub2))
    result[index].append(xor)

print(b"".join(b for a, b, c in result))